In [65]:
import tensorflow as tf
import tensorflow_datasets
from tensorflow.python import keras

In [66]:
# loading the data
mnist = tensorflow_datasets.load('mnist')

In [67]:
# variables used
# input details
input_width = 28
input_height = 28
input_channels = 1
input_pixels = 784

# layer details - units, filter sizes, stride
n_conv1 = 32
n_conv2 = 64
n_hidden = 1024
n_out = 10

conv1_k = conv2_k = 5
max_pool1_k = max_pool2_k = 2

stride_conv1 = stride_conv2 = 1

input_size_to_hidden = (input_width//(max_pool1_k * max_pool2_k)) * (input_height//(max_pool1_k * max_pool2_k)) * n_conv2

In [68]:
# initializing weights and biases
weights = {
    "wc1" : tf.Variable(tf.random.normal([conv1_k, conv1_k, input_channels, n_conv1])),
    "wc2" : tf.Variable(tf.random.normal([conv2_k, conv2_k, n_conv1, n_conv2])),
    "wh1" : tf.Variable(tf.random.normal([input_size_to_hidden, n_hidden])),
    "wo" : tf.Variable(tf.random.normal([n_hidden, n_out]))
}

biases = {
    "bc1" : tf.Variable(tf.random.normal([n_conv1])),
    "bc2" : tf.Variable(tf.random.normal([n_conv2])),
    "bh1" : tf.Variable(tf.random.normal([n_hidden])),
    "bo" : tf.Variable(tf.random.normal([n_out]))
}

In [72]:
# Convlution layer
"""
1. Apply filter to the image -> conv2d()
conv2d(input, weights, biases, padding_type, strides = [no_of_examples, filter_width, filter_height, input_channel])
All the times, no_of_examples and input_channel is set to 1.
So, all the images will get processed 1 by 1 without skipping.
input channel indicates filter_depth.

2. Adding bias values -> bias_add()
3. Applying relu activation function
"""
def conv(x, weights, biases, strides = 1):
    out = tf.nn.conv2d(x, weights, padding = "SAME", strides = [1, strides, strides, 1])
    out = tf.nn.bias_add(out, biases)
    out = tf.nn.relu(out)
    
    return out

# Max pooling layer
"""
Applying pooling to the image -> max_pool()
max_pool(input, padding_type, ksize = [no_of_images, pooling_filter_width, pooling_filter_height, input_channel], strides = [no_of_examples, filter_width, filter_height, input_channel])
ksize argument -> pooling_filter_size
no_of_images and input_channel argument is always set to 1.
So, that the filter cover fills up the entire space completely including the depth.
"""
def maxpooling(x, k = 2):
    return tf.nn.max_pool(x, padding = "SAME", ksize = [1, k, k, 1], strides = [1, k, k, 1]) 

In [120]:
# forward propogation
"""
1. Reshape the image from 784 pixels to 28*28*1 (1 -> input channel) 1D to 2D
2. pass step 1 o/p through convolution layer 1 and max pooling 1
3. pass step 2 o/p through convolution layer 2 and max pooling 2
4. Reshape the step 3 o/p image from (7*7*64) to 3136 pixels (1 -> input channel) 2D to 1D and pass it through hidden layer with relu activation
5. Pass step 4 o/p through output layer with identity activation function 
"""
def cnn(x, weights, biases, keep_prob):
    x = tf.reshape(x, [-1, input_width, input_height, input_channels])
    
    conv1 = conv(x, weights["wc1"], biases["bc1"], stride_conv1)
    conv1_pool = maxpooling(conv1, max_pool1_k)
    
    conv2 = conv(conv1_pool, weights["wc2"], biases["bc2"], stride_conv2)
    conv2_pool = maxpooling(conv2, max_pool2_k)
    
    hidden_input = tf.reshape(conv2_pool, shape = [-1, input_size_to_hidden])
    hidden_output_before_activation = tf.add(tf.matmul(hidden_input, weights["wh1"]), biases["bh1"])
    hidden_output_before_dropout = tf.nn.relu(hidden_output_before_activation)
    hidden_output = tf.nn.dropout(hidden_output_before_dropout, keep_prob)
    
    output = tf.add(tf.matmul(hidden_output, weights["wo"]), biases["bo"])
    
    return output

In [122]:
# Splitting into input and output
x = tf.compat.v1.placeholder("float", [None, input_pixels])
y = tf.compat.v1.placeholder(tf.int32, [None, n_out])
keep_prob = tf.compat.v1.placeholder("float")
                
pred = cnn(x, weights, biases, keep_prob)

In [123]:
# cost function
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = pred, labels = y))

In [124]:
# optimization
optimizer = tf.optimizers.Adam(learning_rate = 0.01)
optimize = optimizer.minimize(cost)

TypeError: minimize() missing 1 required positional argument: 'var_list'

In [125]:
sess = tf.Session()
sess.run(tf.compat.v1.global_variables_initializer())

AttributeError: module 'tensorflow' has no attribute 'Session'